In [ ]:
import os
import io
import zipfile
import textwrap
from typing import Dict, Generator, List, Optional, Tuple
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key = 'ollama')

In [ ]:
MODEL_CHOICES = [
    "openai/gpt-4o-mini",
    "openai/gpt-4o",
    "openai/gpt-3.5-turbo",
    "ollama/llama3.2",
    "ollama/phi3:mini",
    "ollama/qwen2.5:3b",
]
DEFAULT_MODEL = "openai/gpt-4o-mini"

In [ ]:
def normalize_model(model: str) -> str:
    return model.split("/", 1)[1]

def get_client(model_choice: str) -> OpenAI:
    if model_choice.startswith("ollama/"):
        return ollama_via_openai
    return client

def stream_response(model_choice: str, messages: List[Dict]) -> Generator[str, None, None]:
    model_name = normalize_model(model_choice)
    api_client = get_client(model_choice)

    try:
        stream = api_client.chat.completions.create(
            model=model_name,
            messages=messages,
            stream=True,
            max_tokens=1500,
            temperature=0.3,
        )
        text = ""
        for chunk in stream:
            delta = chunk.choices[0].delta.content or ""
            text += delta
            yield text
    except Exception as e:
        yield f"Error while streaming from {model_choice}: {str(e)}"

In [ ]:
SYSTEM_PROMPT = """You are a senior Python engineer. Be conservative and helpful.
When asked to annotate code, add clean Google-style docstrings and minimal comments.
When asked to generate tests, write readable pytest test modules.
When asked to create a trading scaffold, make a working 3-file setup with backtesting.
Avoid clever tricks, prioritize clarity and correctness."""

DOC_PROMPT = """Task: Add docstrings and helpful inline comments to this code.
Do not change any logic or flow.
Use {style}-style docstrings. Add type hints: {add_types}.
Return only the updated code."""

TEST_PROMPT = """Task: Generate a pytest test file for this code.
Include tests for normal, edge, and error conditions.
Use plain pytest (no unittest). Add minimal mocks if needed."""

In [ ]:
def make_trading_files(strategy_brief: str, symbol: str) -> Dict[str, str]:
    """Return dictionary of files for a simple strategy, broker, and backtest."""
    strategy_py = f'''"""
strategy.py
Auto-generated strategy module for {symbol}.
Brief: {strategy_brief}
"""
def decide(state, bar):
    """Example SMA crossover."""
    prices = state.setdefault("prices", [])
    prices.append(bar["close"])
    if len(prices) < 20:
        return "HOLD", state
    short = sum(prices[-5:]) / 5
    long = sum(prices[-20:]) / 20
    action = "BUY" if short > long and not state.get("pos") else "SELL" if short < long and state.get("pos") else "HOLD"
    state["pos"] = action == "BUY" or (state.get("pos") and action != "SELL")
    return action, state
'''

    broker_py = """\"\"\"sim_broker.py
Simple in-memory broker simulator.\"\"\"
def init(cash=10000.0):
    return {"cash": cash, "pos": 0, "equity": [], "trades": []}

def execute(state, action, price, size=1):
    if action == "BUY" and state["cash"] >= price * size:
        state["cash"] -= price * size
        state["pos"] += size
        state["trades"].append(("BUY", price))
    elif action == "SELL" and state["pos"] >= size:
        state["cash"] += price * size
        state["pos"] -= size
        state["trades"].append(("SELL", price))
    state["equity"].append(state["cash"] + state["pos"] * price)
    return state
"""

    backtest_py = f'''"""
backtest.py
Run a synthetic backtest for {symbol}.
"""
import random, strategy, sim_broker

def synthetic_data(n=250, start=100.0):
    price = start
    data = []
    for _ in range(n):
        price *= 1 + random.uniform(-0.01, 0.01)
        data.append({{"close": price}})
    return data

def run():
    bars = synthetic_data()
    state = {{"prices": []}}
    broker = sim_broker.init()
    for bar in bars:
        action, state = strategy.decide(state, bar)
        broker = sim_broker.execute(broker, action, bar["close"])
    eq = broker["equity"][-1] if broker["equity"] else broker["cash"]
    print(f"Final equity: {{eq:.2f}} | Trades: {{len(broker['trades'])}}")

if __name__ == "__main__":
    run()
'''

    readme = f"""# Trading Scaffold for {symbol}
Generated from your brief: {strategy_brief}
Files:
- strategy.py — core logic
- sim_broker.py — in-memory execution
- backtest.py — synthetic backtest
Run with:
```bash
python backtest.py
```"""
    return {
        "strategy.py": strategy_py,
        "sim_broker.py": broker_py,
        "backtest.py": backtest_py,
        "README.md": readme,
    }

In [ ]:
def zip_trading_files(files: Dict[str, str]) -> Tuple[str, bytes]:
    buf = io.BytesIO()
    with zipfile.ZipFile(buf, "w", zipfile.ZIP_DEFLATED) as z:
        for name, content in files.items():
            z.writestr(name, content)
    buf.seek(0)
    return "trading_scaffold.zip", buf.read()

In [ ]:
def docstrings_stream(model_choice: str, code: str, style: str, add_types: bool):
    if not code.strip():
        yield "Please paste Python code first."
        return
    sys = {"role": "system", "content": SYSTEM_PROMPT}
    usr = {
        "role": "user",
        "content": DOC_PROMPT.format(style=style, add_types=add_types) + "\n\n" + code,
    }
    yield from stream_response(model_choice, [sys, usr])

In [ ]:
def tests_stream(model_choice: str, code: str):
    if not code.strip():
        yield "Please paste Python code first."
        return
    sys = {"role": "system", "content": SYSTEM_PROMPT}
    usr = {"role": "user", "content": TEST_PROMPT + "\n\n" + code}
    yield from stream_response(model_choice, [sys, usr])

In [ ]:
def trading_scaffold(strategy_brief: str, symbol: str):
    if not symbol.strip():
        symbol = "AAPL"
    files = make_trading_files(strategy_brief or "Simple SMA crossover", symbol)
    name, data = zip_trading_files(files)
    zip_path = "trading_scaffold.zip"
    with open(zip_path, "wb") as f:
        f.write(data)
    return (
        files["strategy.py"],
        files["sim_broker.py"],
        files["backtest.py"],
        files["README.md"],
        zip_path,
    )

In [ ]:
with gr.Blocks(title="DevLab Assistant") as demo:
    gr.Markdown("# DevLab Assistant")
    gr.Markdown(
        "This mini-lab helps with everyday coding tasks:\n"
        "* Add docstrings and helpful comments to existing code\n"
        "* Generate pytest unit tests automatically\n"
        "* Scaffold a small trading simulator to experiment with strategy ideas\n\n"
        "Select a model (OpenAI or a local Ollama one) and try each tab."
    )

    with gr.Tab("Docstrings / Comments"):
        model1 = gr.Dropdown(MODEL_CHOICES, value=DEFAULT_MODEL, label="Model")
        style = gr.Radio(["google", "numpy"], value="google", label="Docstring style")
        add_types = gr.Checkbox(value=False, label="Add basic type hints")
        code_input = gr.Textbox(
            lines=14,
            label="Paste your Python code here",
            placeholder="def multiply(a, b):\n    return a * b",
        )
        output_md = gr.Markdown(label="Result")
        gen_btn = gr.Button("Generate Docstrings")

        gen_btn.click(
            fn=docstrings_stream,
            inputs=[model1, code_input, style, add_types],
            outputs=output_md,
        )

    with gr.Tab("Unit Tests"):
        model2 = gr.Dropdown(MODEL_CHOICES, value=DEFAULT_MODEL, label="Model")
        code_input2 = gr.Textbox(
            lines=14,
            label="Paste the code you want tests for",
            placeholder="class Calculator:\n    def add(self, a, b):\n        return a + b",
        )
        output_md2 = gr.Markdown(label="Generated test file")
        gen_btn2 = gr.Button("Generate Tests")

        gen_btn2.click(
            fn=tests_stream,
            inputs=[model2, code_input2],
            outputs=output_md2,
        )

    with gr.Tab("Trading Scaffold"):
        gr.Markdown(
            "Generate a minimal, self-contained trading simulator that includes:\n"
            "* `strategy.py`: basic SMA crossover strategy\n"
            "* `sim_broker.py`: in-memory broker\n"
            "* `backtest.py`: synthetic data backtest\n"
            "You can run it locally with `python backtest.py`."
        )

        brief = gr.Textbox(
            lines=6,
            label="Strategy Brief",
            placeholder="e.g., SMA crossover with fast=5, slow=20, long-only",
        )
        symbol = gr.Textbox(value="AAPL", label="Symbol")
        gen_btn3 = gr.Button("Generate Scaffold")

        s_md = gr.Code(language="python", label="strategy.py")
        b_md = gr.Code(language="python", label="sim_broker.py")
        bt_md = gr.Code(language="python", label="backtest.py")
        r_md = gr.Markdown(label="README.md")
        zip_out = gr.File(label="Download ZIP")

        gen_btn3.click(
            fn=trading_scaffold,
            inputs=[brief, symbol],
            outputs=[s_md, b_md, bt_md, r_md, zip_out],
        )

    gr.Markdown("---")
    gr.Markdown(
        "Tips:\n"
        "* Ollama models must be pulled locally (for example `ollama pull llama3.2`).\n"
        "* OpenAI models require the `OPENAI_API_KEY` environment variable.\n"
        "* Everything runs safely and offline for the local models."
    )

In [ ]:
if __name__ == "__main__":
    demo.launch()